<a href="https://colab.research.google.com/github/Kerriea-star/Advanced/blob/master/Customization_basics_tensors_and_ooperations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import TensorFlow**

As of TensorFlow 2, eager execution is turned on by default. Eager execution enables a more interactive frontend to TensorFlow.

In [7]:
import tensorflow as tf

**Tensors**

A Tensor is a multi-dimensional array. Similar to Numpy ndarray objects, tf.Tensor objects have a data type and a shape. Additionally, tf.Tensor s can reside in accelerator memory (like GPU). TensorFlow offers a rich library of operations (for example, tf.math.add, tf.linalg.matmul and tf.linalg.inv) that consume and produce tf.Tensors. These operations automatically convert built-in Python types. For example:

In [8]:
print(tf.math.add(1, 2))
print(tf.math.add([1, 2], [3, 4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.math.square(2) + tf.math.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


Each tf.Tensor has a shape and a datatype:

In [9]:
x = tf.linalg.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


The most obvious difference between Numpy arrays and tf.Tensors are:

1.   Tensors can be backed by accelerator memory (like GPU, TPU)
2.   Tensors are immutable

**Numpy compatibility**

Converting between a TensorFlow tf.Tensor and a Numpy ndarray is easy:

*   TensorFlow operations automatically convert Numpy ndarrays to Ternsors
*   Numpy operations automatically convert Tensors to Numpy ndarrays.


Tensors are explicitly converted to Numpy ndarrays using their .numpy() method. These conversions are typically cheap since the array and tf.Tensor share the underlying memory representation, if possible. However, share the underlying representation isn't always possible since the tf.Tensor may be hosted in GPU memory while Numpy arrays are always backed by host memory, and conversion involves a copy from GPU to host memory.



In [10]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.math.multiply(ndarray, 42)
print(tensor)

print("And Numpy operations convert tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And Numpy operations convert tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


**GPU acceleration**

Many TensorFlow operations are accelerated using the GPU for computation. Without any annotations, TensorFlow automatically decided whether to use the GPU or CPU for an operation-copying the tensor between CPU and GPU memory, if necessary. Tensors produced by an operation are typically backed by the memory of the devide on which the operation executed. For example:

In [11]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0: "),
print(x.device.endswith("GPU:0"))

Is there a GPU available: 
[]
Is the Tensor on GPU #0: 
False


**Device names**

The Tensor.devices property provides a fully qualified string name of the device hosting the contents of the tensor. This name encodes many details such as an identifier of the network address of the host on which this program is executing and the device within that host. This is required for distributed execution of a TensorFlow program. The string ends with with GPU:<N> if the tensor is placed on the N-th GPU on the host.

**Explicit device placement**

In TensorFlow, placement refers to how individual operations are assigned (place on) a device for execution. As mentioned, when there is no explicit guidance provided, tensorflow automatically decides which device to execute an operation and copies tensors to that device, if needed.

However, TensorFlow operations can be explicitly placed on specific devices using the tf.device context manager. For example:

In [12]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.linalg.matmul(x, x)

  result = time.time() - start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("On GPU:")

On CPU:
10 loops: 393.56ms


**Datasets**

This section uses the tf.data.Dataset API to build a pipeline for feeding data to your model. tf.data.Dataset is used to build performant, complex input pipelines form simple, re-usable pieces that will feed your model's training or evaluation to loops.

**Create a source Dataset**

Create a source dataset using one of the factory functions like tf.data.Dataset.from_tensors, tf.data.Dataset.from_tensor_slices, or using objects that read files like tf.data.TextLineDataset or tf.data.TFRecordDataset.

In [13]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line1
  Line 2
  Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)